In [3]:
import numpy as np
import cv2
import os
import random
from skimage.io import imread
from skimage.transform import resize
from skimage import feature
from skimage import exposure
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import category_encoders as ce
from sklearn import naive_bayes
import pandas as pd
from sklearn import metrics 
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.base import clone
import matplotlib.pyplot as plt

### Variables y funciones programadas

In [4]:
#Parametros de raiz
raiz=os.getcwd()
print(raiz)

C:\Users\ivana\Desktop\Universidad\4º curso\1º semestre\Vision artificial\Practicas\Practica final\VisionArtificial


In [5]:
#Tamaño de ventana para homogeneizar las imagenes
tamanoVentana = 100

In [6]:
def tratarImagenes(directorio, X_Train, Y_Train, X_Test, Y_Test,num):
    os.chdir(raiz+directorio)
    aux = os.listdir()
    cont = len(aux) * 0.8
    k = 0
    for j in aux:
        img = cv2.imread(j)

        # Convertimos a escala de grises
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        #Aplicamos el filtro de la mediana que es el mejor para eliminar el ruido sal y pimienta
        img = cv2.medianBlur(img,5)
        
        #Aplicamos el filtro gaussiano para eliminar posibles ruidos restantes
        img = cv2.GaussianBlur(img,(5,5),0)

        #Redimensionamos las imagenes para tener en todas el mismo tamaño
        imagen_ampliada = cv2.resize(img, (tamanoVentana,tamanoVentana))
        
        # Detectamos los bordes con Canny
        canny = cv2.Canny(imagen_ampliada, 50, 150)
        
 
        # Buscamos los contornos
        (contornos,_) = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
        cv2.drawContours(imagen_ampliada,contornos,-1,(0,0,255), 2)
        
        if(k<cont):
            X_Train.append(imagen_ampliada)
            Y_Train.append(num)  
        else:
            X_Test.append(imagen_ampliada)
            Y_Test.append(num)  
        k=k+1

In [7]:
#Parametros para el HOG
pixels_per_cell=(8,8)
cells_per_block=(2,2)
orientations = 9

In [8]:
def HOG(images, pixels_per_cell, cells_per_block):
    lista_hog=[]
    lista_img_hog=[]
    for i,img in enumerate(images):
        carac,img_hog=hog(img, orientations=9, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block,visualize=True, multichannel=False)
        lista_hog.append(carac)
        lista_img_hog.append(img_hog)
    return np.copy(lista_hog),lista_img_hog

# Fase 1: Entrenamiento de un clasificador de monedas

## Fase 1.1: Análisis del conjunto de datos

In [13]:
#Cargamos la imagen y la procesamos para eliminar el ruido
X_Train = []
Y_Train = []
X_Test = []
Y_Test = []

tratarImagenes('\\train\\1c',X_Train, Y_Train, X_Test, Y_Test,1)
tratarImagenes('\\train\\1e',X_Train, Y_Train, X_Test, Y_Test,2)
tratarImagenes('\\train\\2c',X_Train, Y_Train, X_Test, Y_Test,4)
tratarImagenes('\\train\\2e',X_Train, Y_Train, X_Test, Y_Test,5)
tratarImagenes('\\train\\5c',X_Train, Y_Train, X_Test, Y_Test,7)
tratarImagenes('\\train\\10c',X_Train, Y_Train, X_Test, Y_Test,0)
tratarImagenes('\\train\\20c',X_Train, Y_Train, X_Test, Y_Test,3)
tratarImagenes('\\train\\50c',X_Train, Y_Train, X_Test, Y_Test,6)


In [8]:
print("Tamaño de train") 
len(X_Train)

Tamaño de train


1041

In [9]:
print("Tamaño de test") 
len(X_Test)

Tamaño de test


256

In [10]:
#comprobando si se ha aplicado bien el filtro y la redimension

cv2.imshow("imagen",X_Train[456])
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("imagen2",X_Test[100])
cv2.waitKey(0)
cv2.destroyAllWindows()

## Fase 1.2: HOG

In [14]:
X_Train_Coin,_ = HOG(X_Train, pixels_per_cell,cells_per_block)
Y_Train_Coin = np.copy(Y_Train)

X_Test_Coin,_ = HOG(X_Test,pixels_per_cell,cells_per_block)
Y_Test_Coin = np.copy(Y_Test)

# Fase 1.3 Entrenamiento


In [16]:
classifiers = [SVC(), LogisticRegression(random_state=0, max_iter=1000), GaussianNB(), DecisionTreeClassifier()]

clf_names = ['SVM', 'Regr Logistica', 'NB Gaussiano', 'Decision Tree']

score_list = []
aux_params = []
best_score = np.NINF

parameters_dict = {'SVM':{'kernel':('linear','rbf'), 'C':[1,10],'gamma':[0.1,0.001]},
                    'Regr Logistica':{'C':[0.01,1,10,1000] },
                    'NB Gaussiano':{'var_smoothing': np.logspace(0, -9,num=10)},
                    'Decision Tree':{'min_samples_split':[2,3,4], 'criterion':['gini','entropy']},
                 }

for i,(clf_aux,clf_name) in enumerate(zip(classifiers,clf_names)):
        clf = clone(clf_aux)
        clf = GridSearchCV(clf,parameters_dict[clf_name], cv=2, n_jobs=-1)
        clf.fit(X_Train_Coin,Y_Train_Coin)
        
        score=round(clf.score(X_Test_Coin,Y_Test_Coin)*100,2)
        score_list.append(score)
        aux_params.append(clf.best_params_)
        if score >= best_score:
            best_score = score
        

### Mejores parámetros para cada clasificador

### SVM

In [17]:
print("Mejores parámetros para SVM") 
print(aux_params[0])

svc = svm.SVC(kernel='rbf',C=10,gamma=0.001,max_iter=1000)
svc.fit(X_Train_Coin,Y_Train_Coin)
acc = svc.score(X_Train_Coin,Y_Train_Coin)
print("Porcentaje de train: ")
print(acc)

Y_Test_Coin = np.asarray(Y_Test)
acc = svc.score(X_Test_Coin,Y_Test_Coin)
print("Porcentaje de test: ")
print(acc)

Mejores parámetros para SVM
{'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Porcentaje de train: 
0.9356388088376562
Porcentaje de test: 
0.65625


### Regresion logística

In [16]:
print("Mejores parámetros para regresion logística") 
print(aux_params[1])

logistic_regression = LogisticRegression(C=1,max_iter=1000)
logistic_regression.fit(X_Train_Coin,Y_Train_Coin)

accRegLog = logistic_regression.score(X_Train_Coin,Y_Train_Coin)
print("Porcentaje de train: ")
print(accRegLog)


Y_Test_Coin = np.asarray(Y_Test)
accRegLogTest = logistic_regression.score(X_Test_Coin,Y_Test_Coin)
print("Porcentaje de test: ")
print(accRegLogTest)


Mejores parámetros para regresion logística
{'C': 1}
Porcentaje de train: 
1.0
Porcentaje de test: 
0.63671875


### NB Gaussiano

In [17]:
print("Mejores parámetros para NB Gaussiano") 
print(aux_params[2])

nb = naive_bayes.GaussianNB(var_smoothing = 0.1)
nb.fit(X_Train_Coin,Y_Train_Coin)
nbscore = nb.score(X_Train_Coin,Y_Train_Coin)
print("Porcentaje de train: ")
print(nbscore)

Y_Test_Coin = np.asarray(Y_Test)
nbscore_test = nb.score(X_Test_Coin,Y_Test_Coin)
print("Porcentaje de test: ")
print(nbscore_test)

Mejores parámetros para NB Gaussiano
{'var_smoothing': 0.01}
Porcentaje de train: 
0.6628242074927954
Porcentaje de test: 
0.4921875


### Decision tree

In [18]:
print("Mejores parámetros para Decision tree") 
print(aux_params[3])

decision_tree = DecisionTreeClassifier(criterion='gini',min_samples_split=3)
decision_tree.fit(X_Train_Coin,Y_Train_Coin)
accDecTree = decision_tree.score(X_Train_Coin,Y_Train_Coin)
print("Porcentaje de train: ")
print(accDecTree)

Y_Test_Coin = np.asarray(Y_Test)
decision_tree_test = logistic_regression.score(X_Test_Coin,Y_Test_Coin)
print("Porcentaje de test: ")
print(decision_tree_test)


Mejores parámetros para Decision tree
{'criterion': 'entropy', 'min_samples_split': 4}
Porcentaje de train: 
0.9731027857829011
Porcentaje de test: 
0.63671875


# Código de generación de csv


In [21]:
def tratarImagenes2(directorio, X_Train):
    os.chdir(raiz+directorio)
    aux = os.listdir()
    for j in aux:
        img = cv2.imread(j)

        # Convertimos a escala de grises
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        #Aplicamos el filtro de la mediana que es el mejor para eliminar el ruido sal y pimienta
        img = cv2.medianBlur(img,5)
        
        #Aplicamos el filtro gaussiano para eliminar posibles ruidos restantes
        img = cv2.GaussianBlur(img,(5,5),0)

        #Redimensionamos las imagenes para tener en todas el mismo tamaño
        imagen_ampliada = cv2.resize(img, (tamanoVentana,tamanoVentana))
        
        X_Train.append(imagen_ampliada)
        

X_public_test = []

tratarImagenes2('\\public_test',X_public_test)

X_public_test_Coin,_ = HOG(X_public_test, pixels_per_cell,cells_per_block)


In [ ]:
svc = svm.SVC(kernel='rbf',C=10,gamma=0.001,max_iter=1000)
svc.fit(X_public_test_Coin,Y_Train_Coin)
acc = svc.score(X_public_test_Coin,Y_Train_Coin)
print(acc)

In [2]:
import csv
fields = ["Id", "Expected"] 
filename = "results.csv"

# writing to csv file 
with open(filename, 'w', newline="") as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile, delimiter=',') 

    # writing the fields 
    csvwriter.writerow(fields) 

    # writing the data rows 
    for i in range(len(test_labels)):
        csvwriter.writerow([test_image_ids[i], test_labels[i]])

NameError: name 'Y_Train' is not defined